# Useful decorators

## classmethod / staticmethod

**@classmethod** заставляет декорируемую функцию оперировать самим классом, а не конкретным его инстансом (не требует создания объекта-инстанса класса) 

**@staticmethod** "отвязывает" декорируемую функцию от ее класса в том смысле, что функция теряет доступ к полям класса и полям инстанса класса. Но зато и не требует создания инстанса для своей работы

In [29]:
class MethodsDemo:
    """
    A sample class for classmethods and staticmethods
    """
    NAME = 'methods_demo'
    
    def __init__(self, nickname='default', *args):
        self.nickname = nickname
        
    def do(self, *args):
#         print('self is', self)
#         print(args)
        return args
    
    def access_fields(self, *args):
        return self.NAME, self.nickname
    
    @classmethod
    def do_classmethod(*args):
        return args
    
    @classmethod
    def access_fields_cls(cls, *args):
        return cls.NAME
    
    @classmethod
    def generate_cls(cls, *args):
        return cls(*args)
    
    @staticmethod
    def do_staticmethod(*args):
        return args

In [30]:
dir(MethodsDemo)

['NAME',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'access_fields',
 'access_fields_cls',
 'do',
 'do_classmethod',
 'do_staticmethod',
 'generate_cls']

### Вызовы

In [4]:
MethodsDemo.do_classmethod('param')

(__main__.MethodsDemo, 'param')

In [5]:
MethodsDemo.do_staticmethod('param')

('param',)

In [13]:
MethodsDemo().do('param')

('param',)

### Доступы к полям

In [15]:
MethodsDemo.access_fields()

TypeError: access_fields() missing 1 required positional argument: 'self'

In [19]:
MethodsDemo().access_fields()

('methods_demo', 'default')

In [23]:
MethodsDemo.access_fields_cls()

'methods_demo'

staticmethod не имеет доступов к полям, classmethod имеет доступ только к полям класса

In [26]:
MethodsDemo.generate_cls('new_nick').access_fields()

('methods_demo', 'new_nick')

In [33]:
MethodsDemo.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': '\n    A sample class for classmethods and staticmethods\n    ',
              'NAME': 'methods_demo',
              '__init__': <function __main__.MethodsDemo.__init__(self, nickname='default', *args)>,
              'do': <function __main__.MethodsDemo.do(self, *args)>,
              'access_fields': <function __main__.MethodsDemo.access_fields(self, *args)>,
              'do_classmethod': <classmethod at 0x7f7e50533510>,
              'access_fields_cls': <classmethod at 0x7f7e505335d0>,
              'generate_cls': <classmethod at 0x7f7e50533690>,
              'do_staticmethod': <staticmethod at 0x7f7e505336d0>,
              '__dict__': <attribute '__dict__' of 'MethodsDemo' objects>,
              '__weakref__': <attribute '__weakref__' of 'MethodsDemo' objects>})

In [34]:
MethodsDemo().__dict__

{'nickname': 'default'}

## property

In [41]:
class PropertyShow:
    def __init__(self, var):
        self._var = var
    
    @property
    def x(self):
        return self._var

In [42]:
cls = PropertyShow(42)

In [47]:
cls.x

42

In [45]:
cls._var

42

In [49]:
class NotPropertyShow:
    def __init__(self, var):
        self._var = var
    
    def x(self):
        return self._var

In [50]:
cls = NotPropertyShow(42)

In [51]:
cls.x()

42

In [53]:
cls._var

42

To understand what's going on, read great SO answer [here](https://stackoverflow.com/questions/17330160/how-does-the-property-decorator-work-in-python)

## class as a decorator

Экземпляры класса могут быть callable. А если функция может быть декоратором, то что мешает тогда классу? Должен быть определен magic метод `__call__`

In [54]:
class ClassDecorator:
    def __init__(self, func):
        self.entry_str = '[decorating inner func]'
        self.function = func
        self.close_str = '[letting the song flow]'

     
    def __call__(self):
        print(self.entry_str)
        self.function()
        print(self.close_str)

 
 
@ClassDecorator
def function():
    print('Vanderlyle crybaby cry...')

In [56]:
function()

[decorating inner func]
Vanderlyle crybaby cry...
[letting the song flow]


Как и в декораторах-функциях, мы тоже можем возвращать значения (а не только использовать print) и делать параметры

Сделаем таймер

In [60]:
import time

class Timer:
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args):
        t_start = time.time()
        res = self.func(*args)
        total_time = time.time() - t_start
        print(f'{self.func.__name__}({args}) -> {res} executed in {total_time:.2f}s')
        return res

In [61]:
@Timer
def slow_func(seconds_to_sleep):
    time.sleep(seconds_to_sleep)
    return 73

In [62]:
slow_func(3)

slow_func((3,)) -> 73 executed in 3.00s


73

## class as decorated

Класс может быть тоже декорирован функцией или классом

In [70]:
def decorator_function(target):

    def decorator_init(self, *args):
        print("Decorator running")
    
    print('I changed __init__ in class')
    target.__init__ = decorator_init

    return target


class Target:

    def __init__(self, *args):
        self.arg_list = args
        print("Target running")


t = Target('one', 1)
t.__dict__

Target running


{'arg_list': ('one', 1)}

In [71]:
@decorator_function
class Target:

    def __init__(self, *args):
        self.arg_list = args
        print("Target running")

I changed __init__ in class


In [72]:
dt = Target('one', 1)
dt.__dict__

Decorator running


{}

# Descriptors

Короткое описание в [доке](https://docs.python.org/3/reference/datamodel.html#invoking-descriptors)

Мы самом деле, мы с вами уже выше познакомились с частным примером такой сущности.

Дескриптор -- это объект с определенными методами `__get__()`, `__set__()` и `__delete__()`

Обычно когда в Python мы хотим получить доступ к атрибуту класса, то сначала мы ищем его в `__dict__` инстанса класса, потом в классе, и далее выше по цепочке наследования.

## Basics

In [74]:
class BestConstant:
    def __get__(self, obj, objtype=None):
        print('Using descriptor')
        return 73

class Number:
    x = 10
    y = BestConstant()  # наш дескриптор

In [79]:
num = Number()

In [80]:
num.x, num.y

Using descriptor


(10, 73)

In [81]:
num.__dict__, Number.__dict__

({},
 mappingproxy({'__module__': '__main__',
               'x': 10,
               'y': <__main__.BestConstant at 0x7f7e2f2de190>,
               '__dict__': <attribute '__dict__' of 'Number' objects>,
               '__weakref__': <attribute '__weakref__' of 'Number' objects>,
               '__doc__': None}))

In [82]:
num.__dict__['y'] = 42

In [83]:
num.y

42

In [84]:
num.__dict__

{'y': 42}

Видим, что мы переписали значение. Дескриптор, у которого определен только get, называется non-data дескриптором

In [85]:
class BestConstantComplete:
    def __get__(self, obj, objtype=None):
        print('using descriptor')
        return obj._y
    
    def __set__(self, obj, value):
        obj._y = value
        
    def __delete__(self, obj):
        del obj._y

class Number:
    y = BestConstantComplete()  # наш дескриптор
    
    def __init__(self, x=10, y=42):
        self.x = x
        self.y = y

In [86]:
num = Number()

In [87]:
num.__dict__

{'x': 10, '_y': 42}

In [89]:
num.y

using descriptor


42

In [90]:
num.y = 73

In [92]:
num.y

using descriptor


73

In [93]:
num.__dict__

{'x': 10, '_y': 73}

In [94]:
num.__dict__['y'] = 100

In [95]:
num.__dict__

{'x': 10, '_y': 73, 'y': 100}

In [97]:
num.y

using descriptor


73

Если у дескриптора определен `__set__`, то Python при попытке достать атрибут по названию, будет доставать сначала дескриптор, даже если в `__dict__` объекта лежит что-то одноименное 

## Closer to Real world

Например, с помощью дескрипторов можно написать логгер запросов к полю класса.

Пример из документации:

In [98]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAgeAccess:

    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info(f'Accessing "age" giving {value}')
        return value

    def __set__(self, obj, value):
        logging.info(f'Updating age to {value}')
        obj._age = value

class Person:

    age = LoggedAgeAccess()             # Descriptor instance

    def __init__(self, name, age):
        self.name = name                # Regular instance attribute
        self.age = age                  # Calls __set__()

    def birthday(self):
        self.age += 1

In [99]:
denis = Person('Denis Belyakov', 23)

INFO:root:Updating age to 23


In [100]:
vars(denis)

{'name': 'Denis Belyakov', '_age': 23}

In [101]:
denis.__dict__

{'name': 'Denis Belyakov', '_age': 23}

In [102]:
denis.age

INFO:root:Accessing "age" giving 23


23

In [103]:
denis.name

'Denis Belyakov'

In [105]:
denis._age

23

## And yes, property

In [106]:
property.__dict__

mappingproxy({'__getattribute__': <slot wrapper '__getattribute__' of 'property' objects>,
              '__get__': <slot wrapper '__get__' of 'property' objects>,
              '__set__': <slot wrapper '__set__' of 'property' objects>,
              '__delete__': <slot wrapper '__delete__' of 'property' objects>,
              '__init__': <slot wrapper '__init__' of 'property' objects>,
              '__new__': <function property.__new__(*args, **kwargs)>,
              'getter': <method 'getter' of 'property' objects>,
              'setter': <method 'setter' of 'property' objects>,
              'deleter': <method 'deleter' of 'property' objects>,
              'fget': <member 'fget' of 'property' objects>,
              'fset': <member 'fset' of 'property' objects>,
              'fdel': <member 'fdel' of 'property' objects>,
              '__doc__': <member '__doc__' of 'property' objects>,
              '__isabstractmethod__': <attribute '__isabstractmethod__' of 'property' objec

    The main motivation for descriptors is to provide a hook allowing objects stored in class variables to control what happens during attribute lookup.

Хорошая [лекция](http://uneex.org/LecturesCMC/PythonIntro2020/11_MiscOOP) на тему

Развернутый [док](https://docs.python.org/3/howto/descriptor.html)

# Inheritance

## Basics

Синтаксис простой

In [107]:
class BetterList(list):
    # поля и методы
    pass

Порядок видимости снизу вверх: объект -> класс -> родительский класс

Для доступа к полям родителя используется `super()`

In [108]:
class Parent:
    def letter(self):
        return 'D'
    
class Child(Parent):
    def letter(self):
        return super().letter() + 'B'

In [109]:
a = Parent()
b = Child()

In [110]:
a.letter()

'D'

In [111]:
b.letter()

'DB'

In [112]:
class Parent:
    def __init__(self, letter='D'):
        self._letter = letter
        
    def get_letter(self):
        return self._letter
    
class Child(Parent):
    def __init__(self, parent_letter='D', child_letter='B'):
        super().__init__(parent_letter)
        print(f'parent inited {self._letter}')
        self.new_letter = child_letter
        
    def get_child_letter(self):
        return self.new_letter

In [113]:
a = Child()

parent inited D


In [114]:
a.get_letter()

'D'

In [115]:
a.get_child_letter()

'B'

In [116]:
a.__dict__

{'_letter': 'D', 'new_letter': 'B'}

## Protecting names

Фичи наследования в доопределении пространства имен и в возможности их перегрузки  

Предполагается, что если пользователь переопредляет имя, значит он этого хочет 

Если создатель класса не хочет, чтобы какое-то поле было явно перегружено, то можно, например, использовать дескрипторы или особый нейминг

Есть общее соглашение, что переменные, названные с `_` не надо трогать вне класса, в котором они объявлены

Также в Python предусмотрен механизм с названием, начинающися с `__`

In [125]:
class StrictParent:
    def __init__(self):
        self.__letter = 'D'

class Child(StrictParent):
    def __init__(self):
        super().__init__()
        self._letter = 'B'

In [126]:
a = Child()

In [127]:
a.__dict__

{'_StrictParent__letter': 'D', '_letter': 'B'}

# Exceptions

Оператор **try-except-else-finally**

In [131]:
a = '123'

try:
    a[0] = '10'
except BaseException:
    print('impossible to modify object')

impossible to modify object


In [129]:
a[0] = '10'

TypeError: 'str' object does not support item assignment

In [132]:
TypeError.mro()

[TypeError, Exception, BaseException, object]

In [141]:
a = [1, 2, 3]
a = '123'

try:
    a[0] = '10'
except Exception as e:
    print('impossible to modify object')
else:
    print('possible to modify object', a)

impossible to modify object


Исключения — не «ошибки», а способ обработки некоторых условий не там, где они были обнаружены.

In [135]:
raise TypeError

TypeError: 

In [136]:
raise TypeError('message')

TypeError: message

Хорошая [лекция](http://uneex.org/LecturesCMC/PythonIntro2020/10_Inheritance) по механизму наследования в т.ч. с примерами исключений 

**Bonus:**

Напишите декоратор-класс, который бы:
1. Проверял входные аргументы и вывод функции на соответствие типам, указанным в аннотациях 
2. Проверял, что все численные типы находятся в диапазоне от -1000 до 1000
3. Длины типов-последовательностей не превосходят 1000
4. В случае ошибок валидации возвращал информативное сообщение об ошибке в виде Exception
5. Мог бы быть отключаемым по параметрам

Для решения используйте дескрипторы, вам очень помогут упомянутые сегодня ссылки 

In [142]:
def func(a: int, b: str) -> str:
    pass

---

# Некоторые моменты по домашкам

**Общие комментарии:**

- Используйте, пожалуйста, форматтер. В jupyter можно поставить через nbextensions, в тоже должен быть
- snake_case, please
- **Думайте над сложностью вашего алгоритма по времени и по памяти!** Самое быстрое в написании решение не всегда самое эффективное. Помните о сложностях операций над разными структурами в Python. Ищите крайние случаи
- Читайте условия внимательнее, там иногда полезные вещи пишут :)


- Если есть желание разобрать задачку, пишите ассистенту или в чат. Можем это сделать либо в конце семинара, либо в виде консультации

## **Найти максимальный отрицательный элемент, который не меньше -2000**

In [251]:
a = [-1 , 5, 9, -20, -25, -50, 214124]

In [144]:
curr_max_value = -2000

for elem in a:
    if elem < 0 and elem > curr_max_value:
        curr_max_value = elem

In [145]:
curr_max_value

-1

Не очень удачное решение с точки зрения потребления памяти + лишний проход за линию

In [146]:
b = []

for elem in a:
    if elem < 0:
        b.append(elem)

max(b)

-1

Красиво, но тоже не самое оптимальное, т.к. кажется есть лишний проход за линию

In [147]:
max(filter(lambda x: x < 0, a))

-1

## Список, в котором элементы упорядочены по неубыванию. Сколько в нем различных элементов?

In [201]:
len_a = 1000000
a = [i for i in range(len_a)]
len(a)

1000000

In [239]:
import sys

def get_size_in_mb(obj):
    return f'{sys.getsizeof(obj) / (1024 * 1024):.2f} Mb'

### Хранение истории в списке и взятие его длины

In [228]:
len_a = 100000
a = [i for i in range(len_a)]

In [229]:
%%time

b = []
cnt = 0

for elem in a:
    if elem not in b:
        b.append(elem)
        
len(b)

CPU times: user 50.9 s, sys: 0 ns, total: 50.9 s
Wall time: 50.9 s


100000

`if elem not in b` -- проход за линию внутри линейного цикла => квадратичная сложность алгоритма от размера входных данных. Если у вас много разных элементов, будет оооочень долго выполняться + лишний оверхед по памяти

С такой сложностью на большем количестве данных слишком долго ждать. Для сравнения со следующими примерами по памяти можно ориентироваться на значение:

In [242]:
len_a = 1000000
a = [i for i in range(len_a)]

print(get_size_in_mb(a))

8.29 Mb


### Хранение истории в set и взятие его длины

In [243]:
len_a = 1000000
a = [i for i in range(len_a)]

In [244]:
%%time

b = set()
cnt = 0

for elem in a:
    if elem not in b:
        b.add(elem)
        
len(b)

CPU times: user 154 ms, sys: 12 ms, total: 166 ms
Wall time: 164 ms


1000000

Сильно лучше по времени, но все еще оверхед по памяти

In [245]:
print(get_size_in_mb(b))

32.00 Mb


### длина множества

In [246]:
len(set(a))

1000000

In [247]:
print(get_size_in_mb(set(a)))

32.00 Mb


### Проход за линию

In [249]:
%%time

breaks = 0
prev = None

for elem in a:
    if prev is None or elem != prev:
        breaks += 1
    prev = elem
    
breaks

CPU times: user 159 ms, sys: 8 ms, total: 167 ms
Wall time: 165 ms


1000000

И без оверхеда по памяти

    Конечно, на такой простой задаче все это может казаться немного излишним, но если вы будете работать с реальными большими данными, помнить о костах по памяти и времени нужно